###**Importing Necessary Libraries**

In [ ]:
!pip install wandb -qU

In [ ]:
import numpy as np
from keras.datasets import fashion_mnist, mnist
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
import wandb

In [ ]:
wandb.login()
wandb.init(project="CS6910 Assignment 1")

###**Loading Data**

In [ ]:
(X, y), (X_test, y_test) = fashion_mnist.load_data()

### **Question 1 - Plotting MNIST Fashion Dataset**

In [ ]:
classes = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
fig, ax = plt.subplots(2,5,figsize=(20, 10))
ax = ax.reshape(-1)

for i in range(10):
  sample = np.array(np.where(y == i)).reshape(-1)
  sample = sample[3]
  image = X[sample]
  ax[i].set_title(classes[i])
  ax[i].imshow(image)
  
plt.show()
wandb.log({'Labels':fig})

###**Helper Functions - 1**

In [ ]:
#One Hot Encoding for y
def one_hot_encode(y):
  encoded_array = np.zeros((y.size, y.max()+1), dtype=int)
  encoded_array[np.arange(y.size),y] = 1 
  return encoded_array

#Converts softmax probabilities to labels
def softmax_to_label(softmax_output):
  max_index = np.argmax(softmax_output, axis = 0)
  return max_index



### **Data Pre-Processing**

In order to train this model, I have set the input vector X to have a size of (784, 60000), whereas initially it was a vector of shape (60000, 28, 28). To achieve this, I utilized the reshape() function and then normalized the input by dividing it by 255.

Likewise, I performed one hot encoding on the label sequence y. However, please note that I did not encode y_test, as it simplifies the accuracy calculation process when evaluating acccuracy on test data.

In [ ]:
X = X.reshape(X.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)


X = X/255
X_test = X_test/255
y = one_hot_encode(y)


X = X.T
X_test = X_test.T
y = y.T

### **Activation Functions**

Here are the 4 activation functions I used for my network. 

While dealing with MNIST/ Fashion-MNIST , output layer will have softmax as it's activation function inorder to output a vector of probabilities.

In [ ]:
def sigmoid(z):
    #print(-z)
    return 1 / (1 + np.exp(-(z)))

def tanh(z):
    return np.tanh(z)

def relu(z):
    return (z>0)*(z) + ((z<0)*0)

def leakyRelu(z):
    return (z>0)*(z) + ((z<0)*(z)*0.01)

def softmax(x):
    # x = np.float128(x)
    temp = np.exp(x-np.max(x, axis = 0))
    fin = temp/temp.sum(axis = 0)
    return fin

### **Weight Initialisation**

Attaching the reference I used for implementing Xavier initialisation: [Xavier Initialisation](https://cs230.stanford.edu/section/4/)

This initialises a dictionary of parameters of length = *hidden layer size + 1*


> **Parameters:** { 'W1' : $W_{(n[0],X)}$ , 'b1' : $b_{(n[0],1)}$ , 'W2' : $W_{(n[1],n[0])}$ , 'b2' : $b_{(n[1],1)}$ , 'W2' : $W_{(n[2],n[1])}$, 'b2' : $b_{(n[2],1)}$ , . . .  .  .  }




where n is the vector of hidden layer sizes.

While dealing with MNIST/ Fashion-MNIST , input size is 784 and output layer size is 10



In [ ]:
def initialize_parameters(input_size, n, output_size, initialisation):
  if (initialisation == 'Random'):
    parameters = {}
    parameters['W'+str(1)] = np.random.randn(n[0],input_size)*0.01
    parameters['b'+str(1)] = np.random.randn(n[0],1)
    for i in range(1,len(n)):
        parameters['W'+str(i+1)] = np.random.randn(n[i],n[i-1])*0.01
        parameters['b'+str(i+1)] = np.random.randn(n[i],1)
    parameters['W'+str(len(n)+1)] = np.random.randn(output_size,n[-1])*0.01
    parameters['b'+str(len(n)+1)] = np.random.randn(output_size,1)

  elif (initialisation == 'Xavier'):
    parameters = {}
    m = np.sqrt(6)/(input_size+n[0])
    parameters['W'+str(1)] = np.random.uniform(-m,m, (n[0],input_size))
    parameters['b'+str(1)] = np.random.randn(n[0],1)
    for i in range(1,len(n)):
        m = np.sqrt(6)/(n[i-1]+n[i])
        parameters['W'+str(i+1)] = np.random.uniform(-m,m, (n[i],n[i-1]) )
        parameters['b'+str(i+1)] = np.random.randn(n[i],1)
    m = np.sqrt(6)/(output_size+n[-1])
    parameters['W'+str(len(n)+1)] = np.random.uniform(-m,m,(output_size,n[-1]))
    parameters['b'+str(len(n)+1)] = np.random.randn(output_size,1)

  return parameters

def initialize_parameters_zeros(input_size, n, output_size):
    parameters = {}
    parameters['W'+str(1)] = np.zeros((n[0],input_size))
    parameters['b'+str(1)] = np.zeros((n[0],1))
    for i in range(1,len(n)):
        parameters['W'+str(i+1)] = np.zeros((n[i],n[i-1]))
        parameters['b'+str(i+1)] = np.zeros((n[i],1))
    parameters['W'+str(len(n)+1)] = np.zeros((output_size,n[-1]))
    parameters['b'+str(len(n)+1)] = np.zeros((output_size,1))
    return parameters

### **Forward Propagation**

This module takes the parameters dictionary, sequence of activation functions, input vector as inputs and outputs a dictionary of layer wise outputs.



> **Layer Wise Outputs:** { 'h1' : $h_{n[0]}$ ,  'a1' : $a_{n[0]}$ , 'h2' : $h_{n[1]}$ ,  'a2' : $a_{n[1]}$ ,  'h3' : $h_{n[2]}$ ,  'a3' : $a_{n[2]}$ , . . .  .  .  }




  Where, h is pre-activation output and a is post-activation output of a particular layer. If g is the activation function, this module basically does,



>$h_i = W_ia_{i-1} + b_i$   

> $g(h_i) = a_i$

In [ ]:
def linear(W, X, b, activation_func):
    #print(f"W Shape = {W.shape}, X Shape= {X.shape}, W= {W}, X = {X}, b = {b} " )
    h = np.matmul(W,X)+b
    if activation_func == 'sigmoid':
        #print(h)
        a = sigmoid(h)
    elif activation_func == 'relu':
        a = relu(h)
    elif activation_func == 'leakyRelu':
        a = leakyRelu(h)
    elif activation_func == 'tanh':
        a = tanh(h)
    elif activation_func == 'softmax':
        a = softmax(h)
    return h,a

def ForwardPropagation(X, parameters, activation_func):
    layer_wise_outputs = {}
    layer_wise_outputs['h1'], layer_wise_outputs['a1'] = linear(parameters['W1'], X, parameters['b1'], activation_func[0])
    for i in range(1, (len(parameters)//2)):
        layer_wise_outputs['h'+str(i+1)], layer_wise_outputs['a'+str(i+1)] = linear(parameters['W'+str(i+1)],layer_wise_outputs['a'+str(i)],parameters['b'+str(i+1)], activation_func[i])
    return layer_wise_outputs

### **Loss, Cost, Accuracy Functions**

Below is the code for mean square error loss, cross entropy loss, cost function, accuarcy score 

Notice that accuracy score takes it's *true values* as labels and *a softmax vector* as predicted values.

In [ ]:
def MSELoss(Y, Y_pred):
    MSE = np.mean((Y - Y_pred) ** 2, axis = 1)
    MSE = np.mean(MSE)
    return MSE

def CrossEntropyLoss(Y, Y_pred):
    CE = [-Y[i] * np.log(Y_pred[i]) for i in range(len(Y_pred))]
    crossEntropy = np.mean(CE)
    return crossEntropy

def cost(Y, Y_pred, loss_func):
    if (loss_func == 'MSE'):
        return (MSELoss(Y, Y_pred))
    elif (loss_func == 'CE'):
        return (CrossEntropyLoss(Y, Y_pred))

def accuracy_score(y_true, y_pred):
    pred_labels = np.argmax(y_pred, axis=0)
    return np.sum(pred_labels == y_true) / len(y_true)


### **Back Propagation**

I have implemented backpropagation with 2 modules in a for loop. First module backpropagates through post-actiavtion outputs(`ActivationBackward`) and second one back propagates through pre-activation outputs(`LayerBackward`). Later the `BackPropagate` function helps us creating the `gradients` dictionary which has all the corresponding gradients of the loss function with respect to each weight and bias.


**Activation Backward:**

This module takes $\frac{dL}{dA_l}$ as `dA`, activation function `g` of that layer as inputs and returns $\frac{dL}{dZ_l}$ as `dZ`



> $\frac{dL}{dZ_l} = \frac{dL}{dA_l} * g'(Z_l)$



**Layer Backward:**

This modules takes $\frac{dL}{dZ_l}$ as `dZ` and $W_l, b_l, a_{l-1}$ as `Wl, bl, A_prev` as inputs and outputs the $\frac{dL}{dA_{l-1}}$ as `dA_prev` along with the gradients with respect to weights,biases of that layer.



> $\frac{dL}{dA_{l-1}} = W_l^T.\frac{dL}{dZ_l}$

> $\frac{dL}{dW_{l}} = (\frac{1}{m})\frac{dL}{dZ_l}.a_{l-1}^T $

> $\frac{dL}{db_{l}} = (\frac{1}{m})\Sigma_{i=1}^n \frac{dL^i}{dZ_l}  $



After computing the gradient of loss with respect to last layer's Z, we can loop through the network by implementing the above modules repeatedly. The main `BackPropagation` function reuturns a dictionary of gradients in the form, 



> **Gradients:** { 'dW1' : $\frac{dL}{dW_{(n[0],X)}}$ , 'db1' : $\frac{dL}{db_{(n[0],1)}}$ , 'dW2' : $\frac{dL}{dW_{(n[1],n[0])}}$ , 'db2' : $\frac{dL}{db_{(n[1],1)}}$ , 'dW2' : $\frac{dL}{dW_{(n[2],n[1])}}$, 'db2' : $\frac{dL}{db_{(n[2],1)}}$ , . . .  .  .  }






In [ ]:
def ActivationBackward(dA, Z, activation_func) :
    
    if (activation_func == 'sigmoid'):
        grad = sigmoid(Z)*(1-sigmoid(Z))
       
    elif (activation_func == 'relu'):
        grad = np.where(Z>0, 1, 0)

    elif (activation_func == 'leakyRelu'):
        grad = np.where(Z>0, 1, 0.01)

    elif (activation_func == 'tanh'):
        grad = 1 - tanh(Z)**2
    elif (activation_func == 'softmax'):
        grad = softmax(Z) * (1-softmax(Z))
    dZ = dA * grad
    return dZ

def softmax_derivative(x):
    return softmax(x) * (1-softmax(x))        
    
def LayerBackward(dZl, Wl, bl, A_prev):
    
    m = A_prev.shape[1]
    # print(m)
    dWl = (1/m) * np.matmul(dZl, A_prev.T)
    dbl = (1/m)* np.sum(dZl, axis=1, keepdims=True)
    dA_prev = np.matmul(Wl.T,dZl)
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dWl.shape == Wl.shape)
    assert (dbl.shape == bl.shape)
    return dWl, dbl, dA_prev
   
def BackPropagate(parameters, layer_wise_outputs,X, Y, activation_func, loss):
    gradients = {}
    l = len(layer_wise_outputs)//2
    m = Y.shape[1]
    AL = layer_wise_outputs['a'+str(l)]
    HL = layer_wise_outputs['h'+str(l)]
    
    if loss == 'CE':
        gradients['dh'+str(l)] = AL-Y
    elif loss == 'MSE':
        gradients['dh'+str(l)] = (AL-Y) * softmax_derivative(HL)
        
    for i in range(l-1,0,-1):
        gradients['dW'+str(i+1)],gradients['db'+str(i+1)],gradients['da'+str(i)] = LayerBackward(gradients['dh'+str(i+1)], parameters['W'+str(i+1)], parameters['b'+str(i+1)], layer_wise_outputs['a'+str(i)])
        gradients['dh'+str(i)] = ActivationBackward(gradients['da'+str(i)], layer_wise_outputs['h'+ str(i)] , activation_func[i-1])
        
    gradients['dW'+str(1)],gradients['db'+str(1)],gradients['da'+str(0)] = LayerBackward(gradients['dh'+str(1)], parameters['W'+str(1)], parameters['b'+str(1)], X)    
    
    return gradients


# parameters = initialize_parameters(2, [1,2,3], 2)
# activation_func = ['sigmoid','sigmoid','sigmoid','softmax']
# X = np.array([1,2]).reshape(2,1)  
# Y = np.array([1,2]).reshape(2,1)
# loss = 'CE'

# layer_wise_outputs = ForwardPropagation(X, parameters, activation_func)
# print(layer_wise_outputs)
# print(BackPropagate(parameters, layer_wise_outputs, X, Y, activation_func, loss))
# print(parameters['W3'].shape)
# print(BackPropagate(parameters, layer_wise_outputs, X, Y, activation_func, loss)['dW3'].shape)

###**Optimisers**

I created a base class for optimizers that includes two attributes: `learning rate` and `weight decay`. Any optimizer can inherit from this class. The child classes differ in their `update` methods, and the update equations for the optimizers implemented are listed below


**References** : [CS6910 Lecture 5](https://iitm-pod.slides.com/arunprakash_ai/cs6910-lecture-5),  [Optimiser Algorithms](https://cs229.stanford.edu/proj2015/054_report.pdf)

To add a new optimiser, we need to write a new iherited class and write the update method accordingly.

###**Regularisation**
I assigned `weight decay` $λ$ as a common attribute to all the optimiser classes. I changed gradients at every update step accordingly.

\begin{align} \nabla 𝓛(w^t) = \nabla L(w^t) + \lambda w_t
\end{align}


For SGD,
\begin{align} w_{t+1} = w_t - \eta \nabla 𝓛(w^t)
\end{align}

Hence,

\begin{align} w_{t+1} = w_t - \eta \left(\nabla L(w^t) + \lambda w_t\right)
\end{align}

where, $\nabla 𝓛(w^t)$ is regularised loss gradient and $\nabla L(w^t)$ is unregularised loss gradient












In [ ]:
class Optimiser:
    def __init__(self, lr, weight_decay):
        self.lr = lr
        self.wd = weight_decay

    def update(self, parameters, gradients):
        raise NotImplementedError

class SGD(Optimiser):
    def __init__(self, lr, weight_decay):
        super().__init__(lr,weight_decay)

    def update(self, parameters, gradients):
        L = len(parameters) // 2 
        for l in range(1, L + 1):
          parameters["W" + str(l)] = (1-self.wd*self.lr)*parameters["W" + str(l)] - self.lr * gradients["dW" + str(l)]
          parameters["b" + str(l)] = (1-self.wd*self.lr)*parameters["b" + str(l)] - self.lr * gradients["db" + str(l)]
        return parameters

class Momentum(Optimiser):
    def __init__(self, lr, beta, weight_decay):
        super().__init__(lr,weight_decay)
        self.beta = beta
        self.v = {}
  

    def update(self, parameters, gradients):
        L = len(parameters)//2
        if self.v == {}:
          for l in range(1, L + 1):
            self.v["W"+str(l)] = 0
            self.v["b"+str(l)] = 0
        for l in range(1, L + 1):
          gradients["dW" + str(l)]= gradients["dW" + str(l)]+self.wd*parameters["W" + str(l)]
          gradients["db" + str(l)] = gradients["db" + str(l)]+self.wd*parameters["b" + str(l)]
          
          self.v["W"+str(l)] = self.beta * self.v["W"+str(l)] + (1-self.beta) * gradients["dW" + str(l)]
          parameters["W" + str(l)] = parameters["W" + str(l)] - self.lr * self.v["W"+str(l)]
          self.v["b"+str(l)] = self.beta * self.v["b"+str(l)] + (1-self.beta) * (gradients["db" + str(l)])
          parameters["b" + str(l)] = parameters["b" + str(l)] - self.lr * self.v["b"+str(l)]
        return parameters

class Nesterov(Optimiser):
    def __init__(self, lr, gamma,weight_decay):
        super().__init__(lr,weight_decay)
        self.gamma = gamma
        self.look_ahead = {}
        self.v = {}
        

    def update(self, parameters, gradients):
        L = len(parameters)//2
        if self.v == {}:
          for l in range(1, L + 1):
            self.v["W"+str(l)] = 0
            self.v["b"+str(l)] = 0
        if self.look_ahead == {}:
          for l in range(1, L + 1):
            self.look_ahead["W"+str(l)] = 0
            self.look_ahead["b"+str(l)] = 0

        for l in range(1, L + 1):
          gradients["dW" + str(l)]= gradients["dW" + str(l)]+self.wd*parameters["W" + str(l)]
          gradients["db" + str(l)] = gradients["db" + str(l)]+self.wd*parameters["b" + str(l)]
            
          self.look_ahead["W"+str(l)] = parameters["W" + str(l)]-self.gamma*self.v["W" + str(l)]
          parameters["W" + str(l)] = self.look_ahead["W"+str(l)] - self.lr * gradients["dW" + str(l)]
          self.v["W"+str(l)] = self.gamma * self.v["W"+str(l)] + self.lr * gradients["dW" + str(l)]

          self.look_ahead["b"+str(l)] = parameters["b" + str(l)]-self.gamma*self.v["b" + str(l)]
          parameters["b" + str(l)] = self.look_ahead["b"+str(l)] - self.lr * gradients["db" + str(l)]
          self.v["b"+str(l)] = self.gamma * self.v["b"+str(l)] + self.lr * gradients["db" + str(l)]

          
        return parameters

class RMSprop(Optimiser):
    def __init__(self, lr, decay_rate, eps,weight_decay):
        super().__init__(lr,weight_decay)
        self.decay_rate = decay_rate
        self.eps = eps
        self.s = {}


    def update(self, parameters, gradients):
        
        L = len(parameters)//2
        if self.s == {}:
          for l in range(1, L + 1):
            self.s["W"+str(l)] = 0
            self.s["b"+str(l)] = 0
        for l in range(1, L + 1):
          gradients["dW" + str(l)]= gradients["dW" + str(l)]+self.wd*parameters["W" + str(l)]
          gradients["db" + str(l)] = gradients["db" + str(l)]+self.wd*parameters["b" + str(l)]
            
          self.s["W"+str(l)] = self.decay_rate * self.s["W"+str(l)] + (1 - self.decay_rate) * (gradients["dW" + str(l)]**2)
          parameters["W" + str(l)] = parameters["W" + str(l)] - self.lr * (gradients["dW" + str(l)]) / (np.sqrt(self.s["W"+str(l)]) + self.eps)

          self.s["b"+str(l)] = self.decay_rate * self.s["b"+str(l)] + (1 - self.decay_rate) * (gradients["db" + str(l)]**2)
          parameters["b" + str(l)] = (1-self.wd)*parameters["b" + str(l)] - self.lr * (gradients["db" + str(l)]) / (np.sqrt(self.s["b"+str(l)]) + self.eps)
        
        return parameters
        

class Adam(Optimiser):
    def __init__(self, lr, beta1, beta2, eps,weight_decay):
        super().__init__(lr,weight_decay)
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.m = {}
        self.v = {}
        self.t = 0

    def update(self, parameters, gradients):
        L = len(parameters)//2
        if self.m == {}:
          for l in range(1, L + 1):
            self.m["W"+str(l)] = 0
            self.m["b"+str(l)] = 0
        if self.v == {}:
          for l in range(1, L + 1):
            self.v["W"+str(l)] = 0
            self.v["b"+str(l)] = 0
        self.t += 1

        for l in range(1, L + 1):
          gradients["dW" + str(l)]= gradients["dW" + str(l)]+self.wd*parameters["W" + str(l)]
          gradients["db" + str(l)] = gradients["db" + str(l)]+self.wd*parameters["b" + str(l)]
            
          self.m["W" + str(l)] = self.beta1 * self.m["W" + str(l)] + (1 - self.beta1) * (gradients["dW" + str(l)])
          self.v["W" + str(l)] = self.beta2 * self.v["W" + str(l)] + (1 - self.beta2) * ((gradients["dW" + str(l)])**2)
          m_hat = self.m["W" + str(l)] / (1 - self.beta1 ** self.t)
          v_hat = self.v["W" + str(l)] / (1 - self.beta2 ** self.t)
          parameters["W" + str(l)] = parameters["W" + str(l)] - self.lr * m_hat / (np.sqrt(v_hat) + self.eps)

          self.m["b" + str(l)] = self.beta1 * self.m["b" + str(l)] + (1 - self.beta1) * (gradients["db" + str(l)])
          self.v["b" + str(l)] = self.beta2 * self.v["b" + str(l)] + (1 - self.beta2) * ((gradients["db" + str(l)])**2)
          m_hat = self.m["b" + str(l)] / (1 - self.beta1 ** self.t)
          v_hat = self.v["b" + str(l)] / (1 - self.beta2 ** self.t)
          parameters["b" + str(l)] = parameters["b" + str(l)] - self.lr * m_hat / (np.sqrt(v_hat) + self.eps)
        
        
        return parameters

 
class Nadam(Optimiser):
    def __init__(self, lr, beta1, beta2, eps,weight_decay):
        super().__init__(lr,weight_decay)
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.m = {}
        self.v = {}
        self.t = 0

    def update(self, parameters, gradients):
        L = len(parameters)//2
        if self.m == {}:
          for l in range(1, L + 1):
            self.m["W"+str(l)] = 0
            self.m["b"+str(l)] = 0
        if self.v == {}:
          for l in range(1, L + 1):
            self.v["W"+str(l)] = 0
            self.v["b"+str(l)] = 0
        self.t += 1

        for l in range(1, L + 1):
          gradients["dW" + str(l)]= gradients["dW" + str(l)]+self.wd*parameters["W" + str(l)]
          gradients["db" + str(l)] = gradients["db" + str(l)]+self.wd*parameters["b" + str(l)]
            
          self.m["W" + str(l)] = self.beta1 * self.m["W" + str(l)] + (1 - self.beta1) * (gradients["dW" + str(l)]-self.wd*parameters["W" + str(l)])
          self.v["W" + str(l)] = self.beta2 * self.v["W" + str(l)] + (1 - self.beta2) * ((gradients["dW" + str(l)]-self.wd*parameters["W" + str(l)])**2)
          m_hat = self.m["W" + str(l)] / (1 - self.beta1 ** self.t)
          v_hat = self.v["W" + str(l)] / (1 - self.beta2 ** self.t)
          m_hat_fin = (self.beta1*m_hat)+(1-self.beta1)*gradients["dW" + str(l)]/(1-(self.beta1)**self.t)
          parameters["W" + str(l)] = (1-self.wd)*parameters["W" + str(l)] - self.lr * (m_hat_fin) / (np.sqrt(v_hat) + self.eps)

          self.m["b" + str(l)] = self.beta1 * self.m["b" + str(l)] + (1 - self.beta1) * (gradients["db" + str(l)]-self.wd*parameters["b" + str(l)])
          self.v["b" + str(l)] = self.beta2 * self.v["b" + str(l)] + (1 - self.beta2) * ((gradients["db" + str(l)]-self.wd*parameters["b" + str(l)])**2)
          m_hat = self.m["b" + str(l)] / (1 - self.beta1 ** self.t)
          v_hat = self.v["b" + str(l)] / (1 - self.beta2 ** self.t)
          m_hat_fin = (self.beta1*m_hat)+(1-self.beta1)*gradients["db" + str(l)]/(1-(self.beta1)**self.t)
          parameters["b" + str(l)] = (1-self.wd)*parameters["b" + str(l)] - self.lr * (m_hat_fin) / (np.sqrt(v_hat) + self.eps)
        
        
        return parameters

### **Training Our Model on Fashion-MNIST**
Trial cell to check the implementation of all the above functions

In [ ]:
# #Model Architechture

# n = [784,[64,32],10]
# activation_func = ['sigmoid','sigmoid','softmax']
# loss = 'MSE'
# batch_size = 200
# learning_rate = 0.001
# # optimiser = SGD(lr = learning_rate)
# # optimiser = Momentum(lr = learning_rate, beta = 0.2)
# # optimiser = Nesterov(lr = learning_rate, gamma = 0.9)
# # optimiser = RMSprop(lr = learning_rate, decay_rate = 0.1,eps = 1e-6)
# # optimiser = Adam(lr = learning_rate, beta1 = 0.9, beta2 = 0.99 ,eps = 1e-6)
# optimiser = Nadam(lr = learning_rate, beta1 = 0.9, beta2 = 0.99 ,eps = 1e-6)


# epochs = 30
# m = X.shape[1]
# parameters = initialize_parameters(n[0],n[1],n[2], 'Random')
# # parameters = parameters_test2
# count = 0

# #loss array 
# # losses = np.array([])

# while count < epochs :
#   training_loss = 0
#   count = count+1
#   l = len(parameters)//2
#   for i in np.arange(0, X.shape[1], batch_size):
#     batch_count = batch_size
#     if i + batch_size > X.shape[1]:
#       batch_count = X.shape[1] - i + 1
#     batch_size = batch_count

#     layer_wise_outputs = ForwardPropagation(X[:,i:i+batch_size], parameters, activation_func)  
#     gradients = BackPropagate(parameters, layer_wise_outputs, X[:,i:i+batch_size], y[:,i:i+batch_size], activation_func, loss)
#     parameters = optimiser.update(parameters, gradients)
#     training_loss = training_loss + cost(y[:,i:i+batch_size], layer_wise_outputs['a'+str(l)],loss)
#   print("Loss after "+ str(count) +"th epoch =" +str(training_loss*(batch_size)/m))



# test_outputs = ForwardPropagation(X_test, parameters, activation_func)


# def softmax_to_label(softmax_output):
#   max_index = np.argmax(softmax_output, axis = 0)
#   return max_index

# test_outputs['a'+str(len(parameters)//2)] = softmax_to_label(test_outputs['a'+str(len(parameters)//2)])


# def accuracy_score(y_true, y_pred):
#     correct = np.sum(y_true == y_pred)
#     total = len(y_true)
#     accuracy = correct / total
#     return accuracy

# # print(softmax_to_label(y_test.T).shape)
# # print(test_outputs['a'+str(len(n))].shape)
# y_test_fin = softmax_to_label(y_test.T)
# print(f"Test Accuracy = {100*accuracy_score( y_test_fin, test_outputs['a'+str(len(parameters)//2)])} %")

# def accuracy_score(y_true, y_pred):
#     pred_labels = np.argmax(y_pred, axis=0)
#     return np.sum(pred_labels == y_true) / len(y_true)

###**Neural Network Class**

Implentation of the class using all the functions generated above. This class has 2 methods. 
####Class Attributes

While initisaling the network, this is the convention I followed. Network Class takes a dictionary N as input which has information of the input size, size of hidden layers, output size, activation function at each layer, weight initialisation.


```
> N = {'n' : [784,[64,64,64],10],
          'activation_func' : ['sigmoid','sigmoid','sigmoid','softmax'],
          'initialisation' : 'Random'
      }
```






####Methods of the class


1.  **Train:**
 
 Fits the model to the given dataset. Takes Train Dataset, Validation Data set, Batch size, Optimiser, Number of epochs as it's input arguments and fits model's parameters to the train data.

 
2.   **Test:**

        Tests the model on Test dataset given as input attributes to this method with the existing parameters.





In [ ]:

class Network():
  def __init__(self, N, log):
    self.n = N['n']
    self.parameters = initialize_parameters(self.n[0],self.n[1],self.n[2], N['initialisation'])
    self.activation_func = N['activation_func']
    self.l = len(self.parameters)//2
    self.log = log

  def train(self,X,y,X_val,y_val,loss, batch_size, optimiser, epochs):
    m = X.shape[1]
    count = 0
    while(count < epochs):
      count = count+1
      training_loss = 0
      training_score = 0
      for i in np.arange(start=0, stop=X.shape[1], step=batch_size):
        batch_count = batch_size
        if i + batch_size > X.shape[1]:
          batch_count = X.shape[1] - i + 1
        layer_wise_outputs = ForwardPropagation(X[:,i:i+batch_size], self.parameters, self.activation_func)  
        gradients = BackPropagate(self.parameters, layer_wise_outputs, X[:,i:i+batch_size], y[:,i:i+batch_size], self.activation_func, loss)
        self.parameters = optimiser.update(self.parameters, gradients)
        training_loss = training_loss + cost(y[:,i:i+batch_size], layer_wise_outputs['a'+str(self.l)],loss)
        training_score = training_score+ accuracy_score(softmax_to_label(y[:,i:i+batch_size]),  layer_wise_outputs['a'+str(self.l)])
        
      training_loss_fin = training_loss/np.ceil(m/batch_size)
      print("Epoch:"+str(count))
      print("Training Loss after "+ str(count) +"th epoch =" +str(training_loss_fin))

      training_score_fin = training_score/np.ceil(m/batch_size)
      print("Training score after "+ str(count) +"th epoch =" + str(100*training_score_fin))

      validation_outputs = ForwardPropagation(X_val, self.parameters, self.activation_func)
      validation_loss = cost(y_val, validation_outputs['a'+str(self.l)],loss)
      print("Validation Loss after "+ str(count) +"th epoch =" + str(validation_loss))
      validation_score = accuracy_score(softmax_to_label(y_val), validation_outputs['a'+str(self.l)])
      print("Validation Score after "+ str(count) +"th epoch =" + str(100*validation_score))
      if(self.log == True):
        metrics = {"train loss": training_loss_fin, "train score": training_score_fin , "val loss": validation_loss, "accuracy": validation_score}
        wandb.log(metrics)


    # def val_test(self,X,Y):
    #   test_outputs = ForwardPropagation(X, self.parameters, self.activation_func)
    #   print(f"Val Accuracy = {100*accuracy_score(softmax_to_label(Y), test_outputs['a'+str(self.l)])} %")
    #   if(self.log == True):
    #     wandb.log({'Val Accuracy' : accuracy_score(softmax_to_label(Y), test_outputs['a'+str(self.l)])})
    
  def test(self,X,Y):
    test_outputs = ForwardPropagation(X, self.parameters, self.activation_func)
    print(f"Test Accuracy = {100*accuracy_score(Y, test_outputs['a'+str(self.l)])} %")
    if(self.log == True):
      wandb.log({'Test Accuracy' : accuracy_score(Y, test_outputs['a'+str(self.l)])})   

In [ ]:
# X_train, X_val, y_train,y_val  = train_test_split(X.T, y.T, test_size=0.1)
# X_train, X_val, y_train,y_val  = X_train.T, X_val.T, y_train.T,y_val.T 

# N = {'n' : [784,[64,64,64],10],
#      'activation_func' : ['sigmoid','sigmoid','sigmoid','softmax'],
#      'initialisation' : 'Random'
#     }


# batch_size = 64
# epochs = 5
# loss = 'CE'
# learning_rate = 1e-3
# # optimiser = SGD(lr = learning_rate, weight_decay = 0.0005)
# # optimiser = Momentum(lr = learning_rate, beta = 0.9, weight_decay = 0)
# # optimiser = Nesterov(lr = learning_rate, gamma = 0.9, weight_decay = 0)
# optimiser = RMSprop(lr = learning_rate, decay_rate = 0.9,eps = 1e-6, weight_decay = 0)
# # optimiser = Adam(lr = learning_rate, beta1 = 0.9, beta2 = 0.99 ,eps = 1e-6, weight_decay = 0)
# # optimiser = Nadam(lr = learning_rate, beta1 = 0.9, beta2 = 0.99 ,eps = 1e-6, weight_decay = 0.0005)


# network = Network(N,False)
# network.train(X_train , y_train, X_val, y_val, loss, batch_size, optimiser, epochs)
# network.test(X_test, y_test)

###**Helper Functions -2** 

In [ ]:
def build_optimiser(optimiser_str,learning_rate,weight_decay):
  if optimiser_str == 'sgd':
    optimiser = SGD(lr = learning_rate, weight_decay=weight_decay)
  elif optimiser_str == 'momentum':
    optimiser = Momentum(lr = learning_rate, beta = 0.9, weight_decay=weight_decay)
  elif optimiser_str == 'nesterov':
    optimiser = Nesterov(lr = learning_rate, gamma = 0.9, weight_decay=weight_decay)
  elif optimiser_str == 'rmsprop':
    optimiser = RMSprop(lr = learning_rate, decay_rate = 0.1,eps = 1e-6, weight_decay=weight_decay)
  elif optimiser_str == 'adam':
    optimiser = Adam(lr = learning_rate, beta1 = 0.9, beta2 = 0.99 ,eps = 1e-6, weight_decay=weight_decay)
  elif optimiser_str == 'nadam':
    optimiser = Nadam(lr = learning_rate, beta1 = 0.9, beta2 = 0.99 ,eps = 1e-6, weight_decay=weight_decay)
  return optimiser


### **Question 4 - Sweeping through Hyper-parameters** 

Implements various sweep methods in wandb.

Report and Results can be found here: [CS6910 Assignment 1- Wandb Experiments and Report ](https://wandb.ai/berserank/CS6910%20Assignment%201)

In [ ]:

X_train, X_val, y_train,y_val  = train_test_split(X.T, y.T, test_size=0.1)
X_train, X_val, y_train,y_val  = X_train.T, X_val.T, y_train.T,y_val.T 


sweep_config = {
    'method': 'random',
    'name' : 'Hyperparameter Tuning-Random Sweep-2'
}

# metric = {
#     'name': 'accuracy',
#     'goal': 'maximize'   
#     }

# sweep_config['metric'] = metric


parameters_dict = {
    'optimiser': {
        'values': ['sgd', 'momentum', 'nesterov', 'rmsprop', 'adam', 'nadam']
        },
    'layer_size': {
        'values': [32,64,128]
        },
    'epochs': {
          'values': [5,10]
        },
    'hidden_layers': {
          'values': [3,4,5]
        },
    'learning_rate': {
          'values': [1e-3,1e-4]
        },
    'weight_initialisation': {
          'values': ['Xavier']
        },
    'activation_functions': {
          'values': ['relu','leakyRelu']
        },
    'batch_size': {
          'values': [16,32,64]
        },
    'weight_decay': {
          'values': [0,0.0001]
        }
    }

sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="CS6910 Assignment 1")


def train_sweep(config=None):
  with wandb.init(config=config) as run:
    config = wandb.config
    
    N = {'n' : [784,config.hidden_layers*[config.layer_size],10],
          'activation_func' : config.hidden_layers*[str(config.activation_functions)]+['softmax'],
          'initialisation' : config.weight_initialisation
        } 

    exp_name = 'e_'+ str(config.epochs)+'_hl_'+str(config.hidden_layers)
    exp_name = exp_name + '_ls_'+ str(config.layer_size)
    exp_name = exp_name+'_bs_'+ str(config.batch_size)+'_ac_'+str(config.activation_functions)+'_optim_'+ config.optimiser+'_in_'+config.weight_initialisation
    
    wandb.run.name = exp_name


    batch_size = config.batch_size
    epochs = config.epochs
    loss = 'CE'
    optimiser = build_optimiser(config.optimiser, config.learning_rate, config.weight_decay)

    network = Network(N,True)
    network.train(X_train , y_train, X_val, y_val, loss, batch_size, optimiser, epochs)
    network.test(X_test, y_test)

wandb.agent(sweep_id, train_sweep, count= 40)
wandb.finish()

###**Question 7 - Plotting the Confusion Matrix**

We achieved best performance with the following metrics:



```
*   Epochs = 10
*   Hidden layers = [64,64,64]
*   Batch Size = 32
*   Activation Function for hidden layers = Leaky ReLU
*   Optimiser = Nadam
*   Learning rate = 1e-3
*   Weight decay = 0
*   Loss = Cross Entropy
*   Weight Initialisation = Xavier
*   Validation Accuracy = 88.88%
```
Plotting the confusion matrix after fitting our network to this model gave 87.1% accuracy on test data




In [ ]:
X_train, X_val, y_train,y_val  = train_test_split(X.T, y.T, test_size=0.1)
X_train, X_val, y_train,y_val  = X_train.T, X_val.T, y_train.T,y_val.T 

N = {'n' : [784,[64,64,64],10],
     'activation_func' : ['leakyRelu','leakyRelu','leakyRelu','softmax'],
     'initialisation' : 'Xavier'
    }


batch_size = 32
epochs = 10
loss = 'CE'
learning_rate = 1e-3
# optimiser = SGD(lr = learning_rate, weight_decay = 0.0005)
# optimiser = Momentum(lr = learning_rate, beta = 0.9, weight_decay = 0)
# optimiser = Nesterov(lr = learning_rate, gamma = 0.9, weight_decay = 0)
# optimiser = RMSprop(lr = learning_rate, decay_rate = 0.9,eps = 1e-6, weight_decay = 0)
# optimiser = Adam(lr = learning_rate, beta1 = 0.9, beta2 = 0.99 ,eps = 1e-6, weight_decay = 0)
optimiser = Nadam(lr = learning_rate, beta1 = 0.9, beta2 = 0.99 ,eps = 1e-6, weight_decay = 0)


network = Network(N,False)
network.train(X_train , y_train, X_val, y_val, loss, batch_size, optimiser, epochs)
network.test(X_test, y_test)

In [ ]:
predicted_outputs = ForwardPropagation(X_test, network.parameters, network.activation_func)
predicted_outputs = softmax_to_label(predicted_outputs['a'+str(len(network.parameters)//2)])
# confusion_matrix = metrics.confusion_matrix(y_test, predicted_outputs)
# cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = classes)
# cm_display.plot()

# plt.show()

fig = ConfusionMatrixDisplay.from_predictions(y_true = y_test, y_pred = predicted_outputs, display_labels = classes, cmap = 'PuBu')
fig = fig.ax_.get_figure() 
fig.set_figwidth(10)
fig.set_figheight(10)  
wandb.log({'Confusion Matrix : Fashion-MNIST':fig})

###**Question 8- MSE Loss Models**

In theory, CE loss is typically used for classification tasks where the output of the model is a probability distribution over a set of classes . CE loss punishes the difference more than MSE loss as the goal is to minimize the difference between the predicted probability distribution and the true probability distribution of the labels. In contrast, MSE loss is typically used for regression tasks where the goal is to minimize the difference between the predicted and true values.


To compare a model using MSE as loss with a model using CE as loss, I am evaluating the performance of two models that have the same hyperparameters, except that one uses cross-entropy loss and the other uses mean squared error (MSE) loss. I am choosing the best models from both the bayesian sweeps for this purpose.  



In [ ]:

X_train, X_val, y_train,y_val  = train_test_split(X.T, y.T, test_size=0.1)
X_train, X_val, y_train,y_val  = X_train.T, X_val.T, y_train.T,y_val.T 


sweep_config = {
    'method': 'bayes',
    'name' : 'Hyperparameter Tuning- Bayesian Sweep - MSE Loss- RL'
}

metric = {
    'name': 'accuracy',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric


parameters_dict = {
    'optimiser': {
        'values': ['sgd', 'momentum', 'nesterov', 'rmsprop', 'adam', 'nadam']
        },
    'layer_size': {
        'values': [32,64,128]
        },
    'epochs': {
          'values': [5,10]
        },
    'hidden_layers': {
          'values': [3,4,5]
        },
    'learning_rate': {
          'values': [1e-3,1e-4]
        },
    'weight_initialisation': {
          'values': ['Random', 'Xavier']
        },
    'activation_functions': {
          'values': ['relu','leakyRelu']
        },
    'batch_size': {
          'values': [16,32,64]
        },
    'weight_decay': {
          'values': [0,0.0005]
        }
    }

sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="CS6910 Assignment 1")


def train_sweep(config=None):
  with wandb.init(config=config) as run:
    config = wandb.config
    
    N = {'n' : [784,config.hidden_layers*[config.layer_size],10],
          'activation_func' : config.hidden_layers*[str(config.activation_functions)]+['softmax'],
          'initialisation' : config.weight_initialisation
        } 

    exp_name = 'e_'+ str(config.epochs)+'_hl_'+str(config.hidden_layers)
    exp_name = exp_name + '_ls_'+ str(config.layer_size)
    exp_name = exp_name+'_bs_'+ str(config.batch_size)+'_ac_'+str(config.activation_functions)+'_optim_'+ config.optimiser+'_in_'+config.weight_initialisation
    
    wandb.run.name = exp_name


    batch_size = config.batch_size
    epochs = config.epochs
    loss = 'MSE'
    optimiser = build_optimiser(config.optimiser, config.learning_rate, config.weight_decay)

    network = Network(N,True)
    network.train(X_train , y_train, X_val, y_val, loss, batch_size, optimiser, epochs)
    network.test(X_test, y_test)

wandb.agent(sweep_id, train_sweep, count= 40)
wandb.finish()

Best accuracy of 82.08% was achieved on validation set with MSE Loss for the following hyper-parameters after performing bayesian sweep over the given hyper parameters.

```
*   Epochs = 5
*   Hidden layers = [128,128,128,128,128]
*   Batch Size = 16
*   Activation Function for hidden layers = Tanh
*   Optimiser = Nadam
*   Learning rate = 1e-4
*   Weight decay = 0
*   Loss = MSE
*   Weight Initialisation = Xavier

```

Running a new experiment with the same parameters but with CE loss and comparing it with MSE loss. Results can be found in the [wandb report](https://wandb.ai/berserank/CS6910%20Assignment%201)

In [ ]:

X_train, X_val, y_train,y_val  = train_test_split(X.T, y.T, test_size=0.1)
X_train, X_val, y_train,y_val  = X_train.T, X_val.T, y_train.T,y_val.T 


sweep_config = {
    'method': 'grid',
    'name' : 'MSE vs CE'
}

parameters_dict = {
    'loss': {
        'values': ['CE', 'MSE']
        }
    }

sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="CS6910 Assignment 1")


def train_sweep(config=None):
  with wandb.init(config=config) as run:
    config = wandb.config
    
    N = {'n' : [784,[128,128,128],10],
          'activation_func' : 3*['sigmoid']+['softmax'],
          'initialisation' : 'Xavier'
        } 

    exp_name = 'e_5_hl_5_ls_128_bs_16_ac_tanh_optim_nadam_in_Xavier_loss_' + config.loss
    wandb.run.name = exp_name


    batch_size = 64
    epochs = 10
    loss = config.loss
    optimiser = Adam(lr = 1e-3, beta1 = 0.9, beta2 = 0.99 ,eps = 1e-6, weight_decay=0)

    network = Network(N,True)
    network.train(X_train , y_train, X_val, y_val, loss, batch_size, optimiser, epochs)
    network.test(X_test, y_test)

wandb.agent(sweep_id, train_sweep, count= 100)
wandb.finish()

###**Question 10- MNIST Dataset**

I am choosing the below three architechtures to test on MNIST dataset: These were the best performing models with a certain activation function. Results can be found in the [wandb report](https://wandb.ai/berserank/CS6910%20Assignment%201)

|  | Architechture 1 | Architechture 2 | Architechture 3|
|----------|----------|----------|----------|
| Epochs | 10 | 10 | 10 |
| Hidden Layers | [64,64,64] | [128,128,128] |  [128,128,128]|
| Batch Size | 32 | 32 | 32|
| Activation function | Leaky ReLU |  Leaky ReLU | ReLU |
| Optimiser | Nadam | Adam | Adam |
| Learning Rate | 1e-3 | 1e-3 | 1e-3 |
| Weight Decay | 0 | 0.0001 | 0.0001 |
| Loss | Cross Entropy | Cross Entropy | Cross Entropy |
| Weight Initialisation | Xavier | Xavier | Xavier |
| Accuracy on Fashion-MNIST | 88.88 | 88.73 | 88.15  |
| Accuracy on MNIST Test data| 96.85 | 97.42| 97.07 |



In [ ]:
(X, y), (X_test, y_test) = mnist.load_data()
X = X.reshape(X.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

def one_hot_encode(y):
  encoded_array = np.zeros((y.size, y.max()+1), dtype=int)
  encoded_array[np.arange(y.size),y] = 1 
  return encoded_array

def softmax_to_label(softmax_output):
  max_index = np.argmax(softmax_output, axis = 0)
  return max_index

X = X.T/255
X_test = X_test.T/255
y = one_hot_encode(y)
y = y.T

In [ ]:
X_train, X_val, y_train,y_val  = train_test_split(X.T, y.T, test_size=0.1)
X_train, X_val, y_train,y_val  = X_train.T, X_val.T, y_train.T,y_val.T 


sweep_config = {
    'method': 'grid',
    'name' : 'Performance check-MNIST Dataset'
}

N1 = {'N': [784,[64,64,64],10], 'activation_func': 'leakyRelu', 'optimiser': 'nadam'} 
N2 = {'N': [784,[128,128,128],10], 'activation_func': 'leakyRelu', 'optimiser': 'adam'} 
N3 = {'N': [784,[128,128,128],10], 'activation_func': 'relu', 'optimiser': 'adam'} 


parameters_dict = {
    'architechture': {
        'values': [N1, N2, N3]
        }
    }

sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="CS6910 Assignment 1")

def train_sweep(config=None):
  with wandb.init(config=config) as run:
    config = wandb.config
    
    N = {'n' : config.architechture['N'],
          'activation_func' : [config.architechture['activation_func']]*3+['softmax'],
          'initialisation' : 'Xavier'
        } 

    exp_name = str(config.architechture)
    wandb.run.name = exp_name


    batch_size = 32
    epochs = 10
    loss = 'CE'
    if (config.architechture['optimiser'] == 'nadam'):
      optimiser = build_optimiser(config.architechture['optimiser'], learning_rate = 1e-3, weight_decay = 0)
    elif (config.architechture['optimiser'] == 'adam'):
      optimiser = build_optimiser(config.architechture['optimiser'], learning_rate = 1e-3, weight_decay = 0.0001)

    network = Network(N,True)
    network.train(X_train , y_train, X_val, y_val, loss, batch_size, optimiser, epochs)
    # network.val_test(X_val,y_val)
    network.test(X_test, y_test)

    #Confusion Matrix
    predicted_outputs = ForwardPropagation(X_test, network.parameters, network.activation_func)
    predicted_outputs = softmax_to_label(predicted_outputs['a'+str(len(network.parameters)//2)])
    fig = ConfusionMatrixDisplay.from_predictions(y_true = y_test, y_pred = predicted_outputs, cmap = 'PuBu')
    fig = fig.ax_.get_figure() 
    fig.set_figwidth(10)
    fig.set_figheight(10)  
    wandb.log({'Confusion Matrix':fig})

wandb.agent(sweep_id, train_sweep)
wandb.finish()